<a href="https://colab.research.google.com/github/brendyn-j-miller/Data_Science_Project/blob/master/FantasyFootballWebScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Import libraries
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from google.colab import files

In [17]:
# Get the input from the user
year = input('What season? (Note: Input a season between 1999 and 2019) ')
week = input('What week of {0} season? '.format(year))

year, week = int(year), int(week)


What season? (Note: Input a season between 1999 and 2019) 2019
What week of 2019 season? 1


In [18]:
# Define webpage URLs
passingURL = """ 
https://www.pro-football-reference.com/play-index/pgl_finder.cgi?request=1&match=game&year_min={year}&year_max={year}&season_start=1&season_end=-1&age_min=0&age_max=99&game_type=A&league_id=&team_id=&opp_id=&game_num_min=0&game_num_max=99&week_num_min={week}&week_num_max={week}&game_day_of_week=&game_location=&game_result=&handedness=&is_active=&is_hof=&c1stat=pass_att&c1comp=gt&c1val=1&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=pass_rating&from_link=1
""".format(year=year, week=week)
receivingURL = """
https://www.pro-football-reference.com/play-index/pgl_finder.cgi?request=1&match=game&year_min={year}&year_max={year}&season_start=1&season_end=-1&age_min=0&age_max=99&game_type=A&league_id=&team_id=&opp_id=&game_num_min=0&game_num_max=99&week_num_min={week}&week_num_max={week}&game_day_of_week=&game_location=&game_result=&handedness=&is_active=&is_hof=&c1stat=rec&c1comp=gt&c1val=1&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=rec_yds&from_link=1
""".format(year=year, week=week)
rushingURL = """
https://www.pro-football-reference.com/play-index/pgl_finder.cgi?request=1&match=game&year_min={year}&year_max={year}&season_start=1&season_end=-1&age_min=0&age_max=99&game_type=A&league_id=&team_id=&opp_id=&game_num_min=0&game_num_max=99&week_num_min={week}&week_num_max={week}&game_day_of_week=&game_location=&game_result=&handedness=&is_active=&is_hof=&c1stat=rush_att&c1comp=gt&c1val=1&c2stat=&c2comp=gt&c2val=&c3stat=&c3comp=gt&c3val=&c4stat=&c4comp=gt&c4val=&order_by=rush_yds&from_link=1
""".format(year=year, week=week)
urls = {
    'Passing': passingURL,
    'Receiving': receivingURL,
    'Rushing': rushingURL
}

In [19]:
# Create dataframes for each stat
dfs = []
for key, url in urls.items():
  response = get(url)
  soup = BeautifulSoup(response.content,'html.parser')
  table = soup.find('table',{'id':'results'})
  df = pd.read_html(str(table))[0]
  df.columns = df.columns.droplevel(level=0)
  df.drop(['Result','Week','G#','Opp','Unnamed: 7_level_1', 'Age', 
           'Rk', 'Lg', 'Date', 'Day'], axis = 1, inplace = True)
  df = df[df['Pos'] != 'Pos']
  df.set_index(['Player', 'Pos', 'Tm'], inplace=True)
  if key == 'Passing':
    df = df[['Yds', 'TD', 'Int', 'Att', 'Cmp']]
    df.rename({'Yds': 'PassingYds', 'Att': 'PassingAtt', 'Y/A': 'Y/PassingAtt', 'TD': 'PassingTD'}, axis = 1, inplace = True)
  elif key =='Receiving':
    df = df[['Rec', 'Tgt', 'Yds', 'TD']]
    df.rename({'Yds': 'ReceivingYds', 'TD': 'ReceivingTD'}, axis = 1, inplace = True)
  elif key == 'Rushing':
    df.drop('Y/A', axis = 1, inplace = True)
    df.rename({'Att': 'RushingAtt', 'Yds': 'RushingYds', 'TD': 'RushingTD'}, axis = 1, inplace = True)
  dfs.append(df)
df = dfs[0].join(dfs[1:], how = 'outer')
df.fillna(0, inplace = True)
df = df.astype('int64')
df.fillna(0, inplace=True)
df = df.astype('int64')

df.head()

,,,PassingYds,PassingTD,Int,PassingAtt,Cmp,Rec,Tgt,ReceivingYds,ReceivingTD,RushingAtt,RushingYds,RushingTD
Player,Pos,Tm,,,,,,,,,,,,
A.J. Brown,WR,TEN,0,0,0,0,0,3,4,100,0,0,0,0
Aaron Jones,RB,GNB,0,0,0,0,0,0,0,0,0,13,39,0
Aaron Rodgers,QB,GNB,203,1,0,30,18,0,0,0,0,3,8,0
Adam Thielen,WR,MIN,0,0,0,0,0,3,3,43,1,0,0,0
Alexander Mattison,RB,MIN,0,0,0,0,0,0,0,0,0,9,49,0


In [20]:
# Calculate the fantasy points for a player
df['FantasyPoints'] = df['PassingYds']/25 + df['PassingTD']*4 - df['Int']*2 + df['Rec'] + df['ReceivingYds']/10 + df['ReceivingTD']*6 + df['RushingYds']/10 + df['RushingTD']*6

df.reset_index(inplace=True)

In [22]:
# Saving and downloading the dataframe
try:
  df.to_csv('season{}_week{}.csv'.format(year, week))
except IndexError:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Uncomment this section if you wish to download the csv file

3files.download('season{}_week{}.csv'.format(year, week))